#### 1. Business Understanding
What business context is the data coming from? What insights would be valuable in that context, and what data would be required for that purporse?

* Define objective or problem definition of the research in respect to company's business and available data

JCPenney is a North American department store that was founded in the year 1902 at Kemmerer Wyoming by James Cash Penney. It's headquaters is at Plano in Texas where it is registered as J.C. Penney Corporation, Inc. The organisation is registered to trade in goods and services. In goods, JCPenney trade in clothing, foot wears for men, children, women (including plus size), home products such as beddings, bath, toiletories, kitchen wares and windows accessories. Other accessories in the collection of their products include handbags, jewelries and beuaty products. For the services, the offer styling salon, optical centers, custom decorating etc. In addition, they are retailers of branded product such as Nike, Levi, Stanford men's tailored clothing etc. To reach its customer base, the organisatin 


has approximately 647 physical stores in America and Puetorico. It maintains an appreciable online presence using e-commerce platform (jcpenney.com, not currently available in UK) through which it is able to market goods and services with a viewing reach of approximate of 26 million people on the average and a staggering 65 million during peak holiday season. They trade in apparels, foot wears including suits, dressess and casual wear for men, women (including plus size). They also sell branded products such as Nike, Levi, stanford men's tailored active clothing etc. In addition, they are dealers in home products such as beddings, bath, toiletories, kichen and windows accessories.
Jewelries, beauty and accesories. 

services include styling salon, optical centers, studios, jewelries.


#### a. Introduction

* How I answered the above questions to create the project

#### b. Importing libraries and loading data

In [8]:
##### import libraries for numerical calculations, data and datetime processing
import numpy as np
import pandas as pd
import datetime as dt

# imported for visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium

#from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import branca.element as be
import branca.colormap as cm

# for processing text data, build model that generates vader score for intensity of customer ratings
import re 
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer # vader score processing
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm.notebook import tqdm

# kmeans is used to cross validate vader output and fine tune the vader score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# to build huggy face ai model and make comparison with other librariew
from transformers import pipeline

import warnings
warnings.filterwarnings("ignore")

In [9]:
#### Load Dataframe
csv_reviews_data = pd.read_csv('data/reviews.csv')
csv_products_data = pd.read_csv('data/products.csv')
csv_user_data = pd.read_csv('data/users.csv')

# The 'lines=True' parameter is used here for json's file to mitigate ValueError: Trailing data
json_reviewers_data = pd.read_json('data/jcpenney_reviewers.json', lines = True)
json_products_data = pd.read_json('data/jcpenney_products.json', lines=True)

In [12]:
states_lat_long_df=pd.read_csv('data/states_data.csv')#Added: Latitude and Longitude data from another source

#### 2. Data understanding and preparation - 
Explore the data and show you understand its structure and relations, with the aid of appropriate visualisation techniques. Assess the data quality, which insights you would be able to answer from it, and what preparation the data would require. Add new data from another source if required to bring new insights to the data you already have.

####  3. **Data modeling (optional)** - Would modeling be required for the insights you have considered? Use appropriate techniques, if so.
**(i). Linear Regression model to show relationship between scores and price**